In [12]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.chat_models import ChatOpenAI

In [2]:
from agents.encounter_designer import Encounter, Action

Action.model_json_schema()

{'description': 'A special move or tactic an enemy may take',
 'properties': {'name': {'default': 'Name of the action',
   'title': 'Name',
   'type': 'string'},
  'description': {'default': 'Description of what the action does and how it works',
   'title': 'Description',
   'type': 'string'}},
 'title': 'Action',
 'type': 'object'}

In [3]:
from langchain_community.utils.openai_functions import convert_pydantic_to_openai_function

In [4]:
convert_pydantic_to_openai_function(Encounter)

{'name': 'Encounter',
 'description': 'Defines an encounter including any enemies, terrain, setting, etc.',
 'parameters': {'$defs': {'Action': {'description': 'A special move or tactic an enemy may take',
    'properties': {'name': {'default': 'Name of the action',
      'title': 'Name',
      'type': 'string'},
     'description': {'default': 'Description of what the action does and how it works',
      'title': 'Description',
      'type': 'string'}},
    'title': 'Action',
    'type': 'object'},
   'Enemy': {'description': 'Defines an enemy including all stats, conditions, and modifiers',
    'properties': {'name': {'default': 'The name of the creature or humanoid enemy',
      'title': 'Name',
      'type': 'string'},
     'stats': {'allOf': [{'$ref': '#/$defs/StatBlock'}],
      'default': 'The stat block for the enemy'},
     'cr': {'default': "The enemy's Challenge Rating",
      'title': 'Cr',
      'type': 'string'},
     'xp': {'default': 'The XP value of the enemy',
      '

In [16]:
from langchain_core.prompts import ChatPromptTemplate
prompt = """
You are designing an encounter for Dungeons and Dragons 5th Edition.  You have the following information:
Input Parameters:

Player Information: Number of players and their respective levels.
Desired Difficulty Level: Easy, Medium, Hard, or Deadly.
Narrative and Setting Context: Brief description of the current narrative and setting in which the encounter will take place.

As the encounter designer you should consider the following:

Select Appropriate Creatures: Choose creatures with Challenge Ratings (CR) that match the desired difficulty level, ensuring that the total encounter difficulty is appropriate for the players' strength.
Encounter Design Principles:
Balance: Ensure that the encounter matches the desired difficulty level.
Narrative Integration: Design the encounter to fit seamlessly into the ongoing story and setting.
Output of encounter details:
Provide a list of creatures or adversaries, including their CRs.
"""

model = ChatOpenAI(temperature=0.0)
parser = JsonOutputFunctionsParser()

In [13]:

chain = (
    ChatPromptTemplate.from_template(prompt)
    | model.bind(functions=[convert_pydantic_to_openai_function(Encounter)])
    | parser
    | RunnablePassthrough.assign(
        xp_values=lambda r: [e["xp"] for e in r["enemies"]]
    )
)

In [14]:
response = chain.invoke({"input": "Design an encounter for three level 1 players that is medium difficulty, and is set in a temple."})

In [15]:
response["total_xp"]

[25, 25]

In [17]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.messages import SystemMessage

input_message = """
Player Information: {players}
Desired Difficulty Level: {difficulty}
Narrative and Setting Context: {setting}
"""

prompt_template = ChatPromptTemplate.from_messages([
    SystemMessage(content=prompt),
    HumanMessagePromptTemplate.from_template(input_message)
])

In [21]:
chain = (
    prompt_template
    | model.bind(functions=[convert_pydantic_to_openai_function(Encounter)])
    | parser
)

In [22]:
chain.invoke({"players": "3 level 1 players", "difficulty": "medium", "setting": "temple"})

OutputParserException: Could not parse function call: 'function_call'